In [11]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [12]:
page = requests.get('https://en.wikipedia.org/wiki/2018_FIFA_World_Cup_squads')
soup = BeautifulSoup(page.content,'html.parser')

#Find all tables in the page
tables = soup.findAll("table")

In [13]:
team_details = soup.find_all('div', attrs={"class":"toc"})
team_list = []
for i in range(0,32):
    team_name = team_details[0].find_all('li',attrs={"class":"toclevel-2"})[i].contents[0].contents[2].contents[0]
    team_list.append(team_name)

In [14]:
team_df = pd.DataFrame(columns=['Team'])
for i in range(0,len(team_list)):
    team_df.loc[i] = team_list[i]
full_df = pd.DataFrame(columns=['Team Name','Player Name','Club Name','Club Link'])

In [15]:
temp_list=[]
player_list = []
team_list = []
for i in range(0,team_df['Team'].size):
    team_name = team_df['Team'][i];
    players = tables[i].find_all('tr',attrs={"class":"nat-fs-player"})
    for j in range(0,23):
        player_name = players[j].find_all('th')[0].contents[0].contents[0]
        club_name = players[j].find_all('td')[5].contents[2].contents[0]
        club_link = players[j].find_all('td')[5].contents[2].get('href')
        club_link = 'https://en.wikipedia.org' + club_link
        temp_list = [team_name,player_name,club_name,club_link]
        player_list.append(temp_list)
#         print(temp_list)

In [16]:
player_df = pd.DataFrame.from_records(player_list,columns=['Nation','Name','Club','Link'])
club_df = player_df[['Club','Link']]
club_links = club_df.Link.unique()
club_list=[]
temp_list = []
for i in range(0,len(club_links)):
    club_page = requests.get(club_links[i])
    club_soup = BeautifulSoup(club_page.content,'html.parser')
    info_table = club_soup.find('table',attrs={"class":"infobox vcard"})
    info_table_rows = info_table.find_all('tr')
    try:
        for j in range(0,12):
            if(info_table_rows[j].contents[0].contents[0]=='League'):
                league_name=(info_table_rows[j].contents[1].contents[0].get('title'))
    except IndexError:
        league_name = 'Unavailable'
        continue
#     print(club_links[i],'--',league_name)
    temp_list = [club_links[i],league_name]
    club_list.append(temp_list)

In [17]:
club_link_df = pd.DataFrame.from_records(club_list,columns=['Link','League'])
player_df = player_df.merge(club_link_df,how='inner',left_on='Link',right_on='Link')
player_df = player_df.drop(['Link'],axis=1)
player_df = player_df.sort_values(by=['Nation','Name','Club'])

In [20]:
player_df.head(5)

,Nation,Name,Club,League
350,Argentina,Cristian Ansaldi,Torino,Serie A
354,Argentina,Federico Fazio,Roma,Serie A
296,Argentina,Gabriel Mercado,Sevilla,La Liga
143,Argentina,Giovani Lo Celso,Paris Saint-Germain,Ligue 1
92,Argentina,Gonzalo Higuaín,Juventus,Serie A


In [21]:
final_series = player_df.groupby('League').count().Name

In [22]:
final_df = final_series.to_frame()

In [23]:
final_df.to_csv('..\\Consolidated Data\\World Cup Data Consolidation.csv')